## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [3]:
%sql
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [5]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [7]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: timestamp (nullable = true)
-- slots: integer (nullable = true)

Facilities Schema
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

Members Schema
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: timestamp (nullable = true)

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [9]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [11]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [13]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03T11:00:00.000+0000,2
1,4,1,2012-07-03T08:00:00.000+0000,2
2,6,0,2012-07-03T18:00:00.000+0000,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [15]:
%sql
select * from facilities limit 10


facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
2,Badminton Court,0.0,15.5,4000,50
3,Table Tennis,0.0,5.0,320,10
4,Massage Room 1,9.9,80.0,4000,3000
5,Massage Room 2,9.9,80.0,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0.0,5.0,450,15
8,Pool Table,0.0,5.0,400,15


In [16]:
%sql
SELECT name FROM facilities
WHERE membercost=0; 

name
Badminton Court
Table Tennis
Snooker Table
Pool Table


####  Q2: How many facilities do not charge a fee to members?

In [18]:
%sql
SELECT COUNT(name)
FROM facilities
WHERE membercost=0;

count(name)
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [20]:
%sql
SELECT facid, name, membercost, monthlymaintenance FROM facilities
WHERE (membercost < monthlymaintenance*0.2);

facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
2,Badminton Court,0.0,50
3,Table Tennis,0.0,10
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80
7,Snooker Table,0.0,15
8,Pool Table,0.0,15


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [22]:
%sql
SELECT * FROM facilities
WHERE facid IN (1,5);

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [24]:
%sql
SELECT name, monthlymaintenance,
  CASE
    WHEN (monthlymaintenance < 100) THEN 'cheap'
    ELSE 'expensive'
  END AS affordability
FROM facilities

name,monthlymaintenance,affordability
Tennis Court 1,200,expensive
Tennis Court 2,200,expensive
Badminton Court,50,cheap
Table Tennis,10,cheap
Massage Room 1,3000,expensive
Massage Room 2,3000,expensive
Squash Court,80,cheap
Snooker Table,15,cheap
Pool Table,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [26]:
%sql
SELECT * FROM members limit 10;

"first(surname, false)"
GUEST


In [27]:
%sql
SELECT surname, firstname, joindate FROM members
ORDER BY joindate DESC
LIMIT 1

surname,firstname,joindate
Smith,Darren,2012-09-26T18:08:45.000+0000


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [29]:
%sql
SELECT * FROM members limit 5;

memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,GUEST,GUEST,GUEST,0,(000) 000-0000,null,2012-07-01T00:00:00.000+0000
1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,null,2012-07-02T12:02:05.000+0000
2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,null,2012-07-02T12:08:23.000+0000
3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,null,2012-07-03T09:32:15.000+0000
4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03T10:25:05.000+0000


In [30]:
%sql
SELECT * FROM facilities limit 5;

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
2,Badminton Court,0.0,15.5,4000,50
3,Table Tennis,0.0,5.0,320,10
4,Massage Room 1,9.9,80.0,4000,3000


In [31]:
%sql
SELECT * FROM bookings limit 5;

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03T11:00:00.000+0000,2
1,4,1,2012-07-03T08:00:00.000+0000,2
2,6,0,2012-07-03T18:00:00.000+0000,2
3,7,1,2012-07-03T19:00:00.000+0000,2
4,8,1,2012-07-03T10:00:00.000+0000,1


In [32]:
%sql
SELECT DISTINCT members.firstname, members.surname, members.memid, facilities.name
FROM members
    INNER JOIN bookings ON members.memid = bookings.memid
    INNER JOIN facilities ON (facilities.facid  = bookings.facid AND facilities.name LIKE 'Tennis Court%')


firstname,surname,memid,name
Timothy,Baker,16,Tennis Court 1
Douglas,Jones,26,Tennis Court 1
Jemima,Farrell,13,Tennis Court 2
Joan,Coplin,22,Tennis Court 1
Ramnaresh,Sarwin,24,Tennis Court 2
David,Jones,11,Tennis Court 2
Tracy,Smith,2,Tennis Court 1
Matthew,Genting,20,Tennis Court 1
Tim,Boothe,8,Tennis Court 1
Jemima,Farrell,13,Tennis Court 1


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [34]:
%sql
SELECT b.facid, b.starttime, f.membercost, f.guestcost
FROM bookings b
LEFT JOIN facilities f ON b.facid = f.facid
WHERE f.membercost < 30 AND f.guestcost < 30 AND b.starttime LIKE '2012-09-14%'

facid,starttime,membercost,guestcost
0,2012-09-14T08:00:00.000+0000,5.0,25.0
0,2012-09-14T10:00:00.000+0000,5.0,25.0
0,2012-09-14T12:30:00.000+0000,5.0,25.0
0,2012-09-14T14:00:00.000+0000,5.0,25.0
0,2012-09-14T16:00:00.000+0000,5.0,25.0
0,2012-09-14T17:30:00.000+0000,5.0,25.0
0,2012-09-14T19:00:00.000+0000,5.0,25.0
1,2012-09-14T08:00:00.000+0000,5.0,25.0
1,2012-09-14T11:00:00.000+0000,5.0,25.0
1,2012-09-14T14:00:00.000+0000,5.0,25.0


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [36]:
%sql
SELECT b.facid, b.starttime, f.membercost, f.guestcost
FROM bookings b
LEFT JOIN 
(
SELECT facid, membercost, guestcost
FROM facilities
WHERE membercost < 30 AND guestcost < 30
) f ON b.facid = f.facid
WHERE b.starttime LIKE '2012-09-14%'

facid,starttime,membercost,guestcost
0,2012-09-14T08:00:00.000+0000,5.0,25.0
0,2012-09-14T10:00:00.000+0000,5.0,25.0
0,2012-09-14T12:30:00.000+0000,5.0,25.0
0,2012-09-14T14:00:00.000+0000,5.0,25.0
0,2012-09-14T16:00:00.000+0000,5.0,25.0
0,2012-09-14T17:30:00.000+0000,5.0,25.0
0,2012-09-14T19:00:00.000+0000,5.0,25.0
1,2012-09-14T08:00:00.000+0000,5.0,25.0
1,2012-09-14T11:00:00.000+0000,5.0,25.0
1,2012-09-14T14:00:00.000+0000,5.0,25.0


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [38]:
%sql
SELECT SUM(
  CASE
  WHEN m.memid = 0 THEN f.guestcost
  ELSE f.membercost
END) as totalrevenue
FROM bookings b
    LEFT JOIN members m ON m.memid = b.memid
    LEFT JOIN facilities f ON f.facid  = b.facid

totalrevenue
42084.70000000035


In [39]:
%sql
SELECT b.memid, b.facid, m.firstname, m.surname, m.memid, f.name, f.guestcost, f.membercost 
FROM bookings b
    LEFT JOIN members m ON m.memid = b.memid
    LEFT JOIN facilities f ON f.facid  = b.facid

count(memid)
4043
